In [1]:
!gdown --id 1OurDQUtbWQacvT32HMqFL7vIUrSMllOp


Downloading...
From: https://drive.google.com/uc?id=1OurDQUtbWQacvT32HMqFL7vIUrSMllOp
To: /content/preprocessed_data.csv
100% 300k/300k [00:00<00:00, 9.50MB/s]


In [2]:
!pip install kaggle

In [7]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d yekenot/fasttext-crawl-300d-2m


mkdir: cannot create directory ‘/root/.kaggle’: File exists
 99% 1.42G/1.44G [00:06<00:00, 238MB/s]
100% 1.44G/1.44G [00:06<00:00, 238MB/s]


In [8]:
!7z e fasttext-crawl-300d-2m.zip -o/content -r



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1545551987 bytes (1474 MiB)

Extracting archive: fasttext-crawl-300d-2m.zip
--
Path = fasttext-crawl-300d-2m.zip
Type = zip
Physical Size = 1545551987

  0%      0% - crawl-300d-2M.vec                          1% - crawl-300d-2M.vec                          2% - crawl-300d-2M.vec                          3% - crawl-300d-2M.vec                          4% - crawl-300d-2M.vec                          5% - crawl-300d-2M.vec                        

In [11]:
# Reading glove vectors in python: https://stackoverflow.com/a/38230349/4084039
def fasttextModel(gloveFile):
    print ("Loading Fasttext Model")
    f = open(gloveFile,'r', encoding="utf8")
    model = {}#for storing word and the corresponding embedding vector for that word
    for line in f:
        splitLine = line.split()#splitting the line and storing it in a list
        word = splitLine[0]#getting the first element and storing it in word
        embedding = np.array([float(val) for val in splitLine[1:]])#obtaining corresponding vector for that word
        model[word] = embedding#storing word as key and embedding vector for that word as value
    print ("Done.",len(model)," words loaded!")
    return model
model = fasttextModel('/content/crawl-300d-2M.vec')

Loading Fasttext Model
Done. 2000000  words loaded!


In [12]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
df=pd.read_csv('preprocessed_data.csv')#creating dataframe using preprocessed_data.csv

In [14]:
df.head(4)


,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n


In [15]:
def preprocess(x):#for removing last character
  x=x[:-1]
  return x

In [16]:
df['source']=df['source'].apply(preprocess)#preprocessing the source data
df['target']=df['target'].apply(preprocess)#preprocessing the target data

In [17]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [18]:
df.shape


(2000, 2)

In [19]:
df=df[df['source'].apply(len)<170]#removing the datapoints where the source sentence length is greater than or equal to 170
df=df[df['target'].apply(len)<200]#removing the datapoints where the target sentence length is greater than or equal to 200

In [20]:
df.shape


(1990, 2)

In [21]:
from sklearn.model_selection import train_test_split
X=df['source']
y=df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.01)#splitting the data in the ratio 99:1
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(1970,)
(20,)
(1970,)
(20,)


Target:

In [22]:
target_tokenizer= Tokenizer()#creating tokenization
target_tokenizer.fit_on_texts(y_train)#fitting on y_train
target_vocab_size= len(target_tokenizer.word_index) + 1#target vocab size
print(len(target_tokenizer.word_index))

3034


In [23]:
target_encoded_docs_train = target_tokenizer.texts_to_sequences(y_train)#converting text to integers
target_encoded_docs_test = target_tokenizer.texts_to_sequences(y_test)#converting text to integers

In [24]:
target_padded_docs_train = pad_sequences(target_encoded_docs_train,padding='post')#padding the sequence with maxlength

In [25]:
target_padded_docs_train.shape


(1970, 43)

In [26]:
target_padded_docs_test = pad_sequences(target_encoded_docs_test,maxlen=target_padded_docs_train.shape[1],padding='post')#padding the sequence with maxlength

In [27]:
target_padded_docs_test.shape


(20, 43)

Source:

In [28]:
source_tokenizer= Tokenizer()#creating tokenziation
source_tokenizer.fit_on_texts(X_train)#fitting on X_train
source_vocab_size= len(source_tokenizer.word_index) + 1#source vocab size
print(len(source_tokenizer.word_index))

3706


In [29]:
source_encoded_docs_train = source_tokenizer.texts_to_sequences(X_train)#converting text to integers
source_encoded_docs_test = source_tokenizer.texts_to_sequences(X_test)#converting text to integers

In [30]:
source_padded_docs_train = pad_sequences(source_encoded_docs_train,padding='post')#padding the sequence with maxlength

In [31]:
source_padded_docs_train.shape

(1970, 39)

In [32]:
source_padded_docs_test = pad_sequences(source_encoded_docs_test,maxlen=source_padded_docs_train.shape[1],padding='post')#padding the sequence with maxlength

In [33]:
source_padded_docs_test.shape

(20, 39)

In [34]:
#we are reshaping the data because sparse_categorical_crossentropy accepts three dimensional

target_padded_docs_train=target_padded_docs_train.reshape((*target_padded_docs_train.shape,1))
target_padded_docs_test=target_padded_docs_test.reshape((*target_padded_docs_test.shape,1))

In [35]:
print(target_padded_docs_train.shape)
print(target_padded_docs_test.shape)

(1970, 43, 1)
(20, 43, 1)


In [36]:
#we are reshaping the data because sparse_categorical_crossentropy accepts three dimensional

source_padded_docs_train=source_padded_docs_train.reshape((*source_padded_docs_train.shape,1))
source_padded_docs_test=source_padded_docs_test.reshape((*source_padded_docs_test.shape,1))

In [37]:
print(source_padded_docs_train.shape)
print(source_padded_docs_test.shape)

(1970, 39, 1)
(20, 39, 1)


In [38]:
#creating an embedding matrix
embedding_matrix = np.zeros((source_vocab_size, 300))
for word, i in source_tokenizer.word_index.items():
    embedding_vector = model.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Model1:

In [45]:
input=tf.keras.layers.Input(shape=(39,))
embed=tf.keras.layers.Embedding(source_vocab_size,300,weights=[embedding_matrix],input_length=source_padded_docs_train.shape[1],trainable=False)(input)

enc=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100))(embed)  
repeat=tf.keras.layers.RepeatVector(43)(enc)

dec=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(repeat)  
dense=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1024, activation='relu'))(dec)
drop=tf.keras.layers.Dropout(0.5)(dense)
output=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(target_vocab_size, activation='softmax'))(drop)

model=tf.keras.models.Model(inputs=input,outputs=output)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 39)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 39, 300)           1112100   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 200)               320800    
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 43, 200)           0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 43, 256)           336896    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 43, 1024)          263168    
_________________________________________________________________
dropout_3 (Dropout)          (None, 43, 1024)          0   

In [46]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [47]:
model.fit(source_padded_docs_train,target_padded_docs_train,batch_size=1024,epochs=50,
          validation_data=(source_padded_docs_test,target_padded_docs_test))

Epoch 1/50
2/2 [==============================] - 6s 2s/step - loss: 7.2103 - accuracy: 0.3273 - val_loss: 4.5655 - val_accuracy: 0.7000
Epoch 2/50
2/2 [==============================] - 1s 417ms/step - loss: 4.4471 - accuracy: 0.6735 - val_loss: 7.5582 - val_accuracy: 0.0186
Epoch 3/50
2/2 [==============================] - 1s 414ms/step - loss: 6.9582 - accuracy: 0.0076 - val_loss: 3.5651 - val_accuracy: 0.7000
Epoch 4/50
2/2 [==============================] - 1s 414ms/step - loss: 3.6384 - accuracy: 0.6735 - val_loss: 3.7226 - val_accuracy: 0.7000
Epoch 5/50
2/2 [==============================] - 1s 418ms/step - loss: 4.4497 - accuracy: 0.6735 - val_loss: 4.3426 - val_accuracy: 0.7000
Epoch 6/50
2/2 [==============================] - 1s 416ms/step - loss: 4.4265 - accuracy: 0.6735 - val_loss: 2.6218 - val_accuracy: 0.7000
Epoch 7/50
2/2 [==============================] - 1s 414ms/step - loss: 3.4221 - accuracy: 0.6735 - val_loss: 2.5167 - val_accuracy: 0.7000
Epoch 8/50
2/2 [=======

In [48]:
x=model.predict(source_padded_docs_test[1:2])[0]

In [49]:
#https://machinelearningmastery.com

index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
index_to_words[0] = '<PAD>'

' '.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])

'i i i i i you you you you you you you you you you <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>'

In [50]:
print(y_test[:1])

443    Hi. Everybody! Me. I'm fine. And wish to chat ...
Name: target, dtype: object


In [51]:
X_test[:1]

443    Hi.......everybody! Me.....i'm fine... ñ wish ...
Name: source, dtype: object

In [52]:
def prediction(x):

  index_to_words = {id: word for word, id in target_tokenizer.word_index.items()}
  index_to_words[0] = '<PAD>'

  y=' '.join([index_to_words[prediction] for prediction in np.argmax(x, 1)])
  return y
for i in range(20):
  print("Input text: ")
  a=list(X_test[i:i+1])
  print(a[0])

  print("Actual Output: ")
  b=list(y_test[i:i+1])
  print(b[0])

  print("Predicted Output: ")
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if i=='<PAD>':
      continue
    else:
      y_lst.append(i)
  print(' '.join(y_lst))
  print('>'*180)

Input text: 
Hi.......everybody! Me.....i'm fine... ñ wish to chat wif ü guys. Can sms to 96792371. K....
Actual Output: 
Hi. Everybody! Me. I'm fine. And wish to chat with you guys. You can message to 96792371. OK.
Predicted Output: 
i i i i i you you you you you you you you you
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
U still wan me 2 reg e gown 4 u? But need ur add, IC n matric. Then e 3 measurement.
Actual Output: 
Do you still want me to register the gown for you? But I will need your address, IC and matriculation number. And also the 3 measurement.
Predicted Output: 
i i i i i you you you you you you you you you you
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Input text: 
You study wat course?
A

In [53]:
import nltk.translate.bleu_score as bleu

bleu_score=[]
for i in range(20):
  b=list(y_test[i:i+1])
  x=model.predict(source_padded_docs_test[i:i+1])
  y=prediction(x[0])
  y=y.split(' ')
  y_lst=[]
  for i in y:
    if i=='<PAD>':
      continue
    else:
      y_lst.append(i)
  bleu_score.append(bleu.sentence_bleu([b[0].split(),],y_lst))
print(bleu_score)
print("The Average Bleu Score is: ",sum(bleu_score)/20)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


[0.3884935863283276, 0.24405885259598942, 0.39011264866539486, 0.41368954504257255, 0, 0, 0.38715386987817624, 0.49432603195143127, 0, 0, 0, 0.4004970149398301, 0, 0.37846125781090306, 0, 0, 0, 0.33125669191122536, 0, 0]
The Average Bleu Score is:  0.1714024749561925
